#### Imports/setup

In [ ]:
from parameters.config_imports import *

In [ ]:
def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

#### Inputs : feature collection (without Geo IDs)
- Choose between 1) GEE asset or 2) Geojson input
- Comment out the one you dont want

In [ ]:
## 1 - GEE asset
asset_id = "projects/ee-whisp/assets/example_asset" # asset id
roi = ee.FeatureCollection(asset_id) # load feature collection asset

## 2 - Geojson input
# geojson_path = "input_examples/geojson_example.geojson" # geojson file
# roi = geojson_to_ee(geojson_path) # convert geojson to feature collection

In [ ]:
roi # view feature collection properties

In [ ]:
# optional code: uncomment below line to test converting to point inputs using polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over 500 polygons currently this will go to Google Drive and require manual uploading here
- This value can be changed in parameters.config_runtime.py 
- NB Currently stats processing sent to a separate notebook 'process_whisp_stats.pynb'

In [ ]:
df = whisp_stats_as_df(roi)

### Display it


In [ ]:
df

#### Export it to CSV file
NB if exporting to Google Drive skip this and upload to your results folder 'whisp_output_table.csv'

In [ ]:
#export output csv (default: '/home/sepal-user/module_results/whisp/whisp_output_table.csv')
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)

##### Calculate EUDR risk category

In [ ]:
# Replace with name of your CSV file 
# NB if skipped 'Add Geo ids' should set to "out_directory/whisp_output_table.csv" 
csv_file_input =out_directory/'whisp_output_table.csv'

# import csv as a dataframe
df_w_stats = pd.read_csv(csv_file_input)

df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


##### Display table with risk columns

In [ ]:
df_w_risk

##### Export table with risk columns to csv 

In [ ]:
output_file=out_directory/'whisp_output_table_w_risk.csv' # edit as required

df_w_risk.to_csv(output_file)


#### Add Geo ids (optional)
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- If processing doesn't complete, just rerun this cell and it should pickup from where you left.
- If registering many features you can also click dollar sign in bottom right of Sepal and under 'Sessions' heading extend the length as needed.

In [ ]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="geoid",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    override_checks=False,
    remove_temp_csv=False,
    debug=False)

##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [ ]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table_w_risk.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [ ]:
# remove_column_from_csv(
#     csv_file="whisp_output_table_w_risk_w_geo_id.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )